# Main script to Join Data on Instrument 2 Fire Location

Modules: N/A <br>
Author: Jordan Meyer <br>
Email: jordan.meyer@berkeley.edu <br>
Date created: Feb 18, 2023 <br>

**Citations (data sources)**


**Citations (persons)**
1. Cornelia Ilin 

**Preferred environment**
1. Code written in Jupyter Notebooks

### Step 1: Import packages

In [70]:
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install geopandas --quiet
    from google.colab import drive

    drive.mount("/content/drive")
    in_dir = (
        in_instrument
    ) = "/content/drive/MyDrive/capstone_fire/notebooks/instrument_2/"
    import drive.MyDrive.capstone_fire.modules.deep_ols as deep_ols
else:
    in_dir = in_instrument = "../data/instrument_2/"
    %cd '..'
    import modules.deep_ols as deep_ols
    %cd 'notebooks'

/Users/jordan/Documents/GitHub/fire_capstone
/Users/jordan/Documents/GitHub/fire_capstone/notebooks


In [71]:
import calendar
import os
from datetime import date, timedelta

# geography
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import shapely

# Moved from sklearn.neighbors to sklearn.metrics following their package change
import sklearn.metrics
from shapely.geometry import Point

# import cartopy.crs as ccrs
# import contextily as ctx
# import fiona
# import netCDF4 as ncdf
# import osmnx as ox
# from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
# from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable


dist = sklearn.metrics.DistanceMetric.get_metric("haversine")

# ignore warnings
import warnings

from tqdm.notebook import tqdm, trange

warnings.filterwarnings("ignore")

### Step 2: Define working directories

### Step 3: Define functions

### Step 4: Read data

## Wind

Wind load from other file

In [72]:
wind_df = (
    (
        pd.read_csv(in_dir + "all_years_wind_data.csv", index_col=0)
        .sort_values(["year_month", "ZCTA10"])
        .reset_index(drop=True)
    )
    .drop("Unnamed: 0", axis=1)
    .rename(columns={"ZCTA10": "ZCTA"})
    .reset_index(drop=True)
    .drop_duplicates()
)


wind_df["year_month"] = wind_df["year_month"].astype(str)
wind_df["ZCTA"] = wind_df["ZCTA"].astype(str)


wind_df[wind_df.ZCTA == "97635"].lat = 41.993
# wind_ready.loc['97635','ZCTA' == '97635']#['lon'] = -120.250
# wind_ready = wind_ready.drop_duplicates()
ind_fix = wind_df[wind_df.ZCTA == "97635"].index
for ind in ind_fix:
    wind_df.iloc[ind, 0] = 41.993
    wind_df.iloc[ind, 1] = -120.250

wind_dates = wind_df["year_month"].unique()
wind_zips = wind_df["ZCTA"].unique()
wind_ready = wind_df
wind_ready

,lat,lon,ZCTA,u,v,wdir,wspd,year_month,year,month
0,37.465,-117.936,89010,0.504258,-0.719008,305.042938,0.878208,199101,1991,1
1,35.396,-116.322,89019,-0.172753,-0.946940,259.661102,0.962568,199101,1991,1
2,36.161,-116.139,89060,-0.435964,-0.812957,241.796738,0.922477,199101,1991,1
3,35.957,-115.897,89061,-0.560538,-1.176384,244.522552,1.303105,199101,1991,1
4,39.520,-120.032,89439,0.042253,0.205661,78.390099,0.209957,199101,1991,1
...,...,...,...,...,...,...,...,...,...,...
627067,39.149,-120.248,96146,0.636599,0.953782,56.279018,1.146716,202212,2022,12
627068,39.236,-120.062,96148,0.636599,0.953782,56.279018,1.146716,202212,2022,12
627069,38.732,-120.033,96150,0.071517,0.472164,81.387100,0.477549,202212,2022,12
627070,39.184,-120.427,96161,-0.217010,0.239125,132.224274,0.322915,202212,2022,12


## Fire

Finsh cleaning some fire things and then join fire to wind 

In [93]:
fire_df = pd.read_csv(in_instrument + "fire_wind_processed.csv", index_col=0)

# Extract year and month, filter for fires in scope 1991+
# fire_df["year"] = fire_df["ALARM_DATE"].str[:4]
fire_df = fire_df[fire_df["year"].astype(int) > 1990]
fire_df = fire_df[fire_df["year"].astype(int) < 2020]

# Extract Month and End Year/Month
fire_df["month"] = fire_df["month"].astype(int)
fire_df["end_year"] = fire_df["CONT_DATE"].str[:4]
fire_df["end_month"] = fire_df["CONT_DATE"].str[5:7].astype(int)

# Give each fire a unique index code for later
fire_df = (
    fire_df.drop_duplicates().reset_index().rename(columns={"Index": "fire_index"})
)

fire_df["geometry"] = gpd.GeoSeries.from_wkt(fire_df["fire_centroid"])
fire_df["lon"] = fire_df["geometry"].apply(lambda p: p.x)
fire_df["lat"] = fire_df["geometry"].apply(lambda p: p.y)

fire_df["sq_mi"] = fire_df["GIS_ACRES"] / 640
fire_df["radius_mi"] = np.sqrt(fire_df["sq_mi"] / np.pi)

# Give each fire a unique index code for later
fire_df = (
    fire_df.drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={"index": "fire_index"})
)

fire_dates = fire_df["year_month"].unique()
fire_indexes = fire_df.index

fire_df.year = fire_df.year.astype(int)
fire_df.month = fire_df.month.astype(int)
fire_df["max_days_in_mo"] = fire_df[["year", "month"]].apply(
    lambda x: calendar.monthrange(*x)[1], axis=1
)

fire_df["sq_mi"] = fire_df["GIS_ACRES"] / 640
fire_df["radius_mi"] = np.sqrt(fire_df["sq_mi"] / np.pi)

fire_df["start_month"] = fire_df["ALARM_DATE"].apply(lambda x: int(x[5:7]))
fire_df["start_day"] = fire_df["ALARM_DATE"].apply(lambda x: int(x[8:10]))

fire_df["end_month"] = fire_df["CONT_DATE"].apply(lambda x: int(x[5:7]))
fire_df["end_day"] = fire_df["CONT_DATE"].apply(lambda x: int(x[8:10]))


fire_df["start_mid_end"] = fire_df[["month", "start_month", "end_month"]].apply(
    lambda x: (
        "complete"
        if x[1] == x[2]
        else ("start" if x[0] == x[1] else ("end" if x[0] == x[2] else "mid"))
    ),
    axis=1,
)

fire_df["fire_days_in_mo"] = fire_df[
    [
        "start_month",
        "end_month",
        "max_days_in_mo",
        "start_day",
        "end_day",
        "start_mid_end",
    ]
].apply(
    lambda x: (
        1
        if (x["end_day"] - x["start_day"] == 0)
        else (
            x["end_day"] - x["start_day"]
            if (x["start_mid_end"] == "complete")
            else (
                x["max_days_in_mo"] - x["start_day"]
                if (x["start_mid_end"] == "start")
                else (
                    x["end_day"]
                    if x["start_mid_end"] == "end"
                    else (x["max_days_in_mo"])
                )
            )
        )
    )
    / x["max_days_in_mo"],
    axis=1,
)

fire_backup = fire_df

fire_ready = fire_df[
    [
        "fire_lat_lon",
        "GIS_ACRES",
        "lat",
        "lon",
        "fire_days_in_mo",
        "fire_wdir",
        "fire_wspd",
        "year_month",
        "year",
        "month",
    ]
]
fire_ready = fire_ready.reset_index(drop=True).rename(
    columns={"GIS_ACRES": "acres", "lat": "fire_lat", "lon": "fire_lon"}
)

fire_ready

,fire_lat_lon,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month
0,"(35.25539508507167, -119.5812970773511)",238.031830,35.255395,-119.581297,0.032258,342.554565,1.595909,199305,1993,5
1,"(39.45165114943642, -120.0093608630314)",19.139362,39.451651,-120.009361,0.032258,0.700786,1.000503,199307,1993,7
2,"(38.63102591471577, -120.8402934705149)",70.179840,38.631026,-120.840293,0.033333,58.361294,0.271346,199309,1993,9
3,"(35.42095813876129, -119.7656494555765)",23.423927,35.420958,-119.765649,0.033333,331.157288,1.023493,199309,1993,9
4,"(35.32190356232154, -118.4169237147862)",151.780167,35.321904,-118.416924,0.032258,7.345666,1.831764,199308,1993,8
...,...,...,...,...,...,...,...,...,...,...
6632,"(37.87494989081461, -119.7281350256987)",331.624695,37.874950,-119.728135,0.166667,73.448456,0.113857,200709,2007,9
6633,"(37.56860595616687, -119.6600328204804)",23.633701,37.568606,-119.660033,0.233333,158.267761,0.253292,200711,2007,11
6634,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,1.000000,215.334213,0.138538,200711,2007,11
6635,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,0.258065,178.254211,0.474964,200712,2007,12


In [94]:
sorted(fire_ready.year_month.unique())

[199101,
 199102,
 199104,
 199106,
 199107,
 199108,
 199109,
 199110,
 199111,
 199112,
 199202,
 199203,
 199205,
 199206,
 199207,
 199208,
 199209,
 199210,
 199211,
 199305,
 199306,
 199307,
 199308,
 199309,
 199310,
 199311,
 199312,
 199401,
 199405,
 199406,
 199407,
 199408,
 199409,
 199410,
 199412,
 199505,
 199506,
 199507,
 199508,
 199509,
 199510,
 199511,
 199512,
 199601,
 199604,
 199605,
 199606,
 199607,
 199608,
 199609,
 199610,
 199611,
 199703,
 199704,
 199705,
 199706,
 199707,
 199708,
 199709,
 199710,
 199804,
 199806,
 199807,
 199808,
 199809,
 199810,
 199901,
 199902,
 199903,
 199904,
 199905,
 199906,
 199907,
 199908,
 199909,
 199910,
 199911,
 199912,
 200001,
 200003,
 200004,
 200005,
 200006,
 200007,
 200008,
 200009,
 200010,
 200011,
 200101,
 200103,
 200105,
 200106,
 200107,
 200108,
 200109,
 200110,
 200111,
 200112,
 200201,
 200202,
 200203,
 200204,
 200205,
 200206,
 200207,
 200208,
 200209,
 200210,
 200211,
 200212,
 200301,
 

## PM2.5

Load PM2.5 from lawis files downloaded from gdrive

In [95]:
aqi1 = pd.read_csv(in_instrument + "jordan_firepm25.csv", index_col=0)
aqi2 = pd.read_csv(in_instrument + "finalpm25.csv", index_col=0)
aqi2

,year_month,ZIP10,pm25
204,2017-01,89010,1.842857
205,2017-02,89010,3.528571
206,2017-03,89010,3.242857
207,2017-04,89010,3.700000
208,2017-05,89010,5.242857
...,...,...,...
585307,2016-12,96148,1.713333
585308,2016-12,96150,1.719288
585309,2016-12,96155,0.771287
585310,2016-12,96161,1.416996


In [96]:
pm25_df = aqi2.reset_index(drop=True).rename(
    columns={"ZIP10": "ZCTA", "year_month": "old_ym"}
)
pm25_df["year_month"] = pm25_df["old_ym"].str[:4] + pm25_df["old_ym"].str[5:]
pm25_df["ZCTA"] = pm25_df["ZCTA"].astype(str)
pm25_ready = pm25_df.drop("old_ym", axis=1)

In [99]:
wfp_df = wind_ready.merge(pm25_ready, on=["year_month", "ZCTA"], how="left")
wind_final = wfp_df.rename(
    columns={
        "lat": "zip_lat",
        "lon": "zip_lon",
        "wdir": "zip_wdir",
        "wspd": "zip_wspd",
        "pm25": "zip_pm25",
    }
).drop(["u", "v"], axis=1)[wfp_df["year"] < 2019]
wind_final.to_csv(in_instrument + "wind_zip_pm25.csv")
wind_final

,zip_lat,zip_lon,ZCTA,zip_wdir,zip_wspd,year_month,year,month,zip_pm25
0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976
1,35.396,-116.322,89019,259.661102,0.962568,199101,1991,1,10.846541
2,36.161,-116.139,89060,241.796738,0.922477,199101,1991,1,12.385000
3,35.957,-115.897,89061,244.522552,1.303105,199101,1991,1,10.150000
4,39.520,-120.032,89439,78.390099,0.209957,199101,1991,1,8.389565
...,...,...,...,...,...,...,...,...,...
548683,39.149,-120.248,96146,76.585587,0.437017,201812,2018,12,4.626316
548684,39.236,-120.062,96148,76.585587,0.437017,201812,2018,12,4.528572
548685,38.732,-120.033,96150,107.369316,0.231856,201812,2018,12,4.286667
548686,39.184,-120.427,96161,151.071350,0.243283,201812,2018,12,5.053509


In [100]:
fire_pm25 = aqi1.rename(
    columns={"lat": "fire_lat", "lon": "fire_lon", "pm25": "fire_pm25"}
).reset_index(drop=True)
fire_pm25.to_csv("fire_pm25.csv")
fire_pm25

,year,month,fire_lat,fire_lon,fire_pm25
0,1991,1,39.155351,-122.775965,0.10
1,1991,1,39.567560,-121.002336,11.24
2,1991,1,39.084997,-122.744482,1.74
3,1991,1,39.756806,-121.331523,10.59
4,1991,1,39.694403,-121.355888,15.02
...,...,...,...,...,...
8267,2016,11,40.375938,-120.751081,3.61
8268,2016,12,34.047386,-118.948054,4.52
8269,2016,12,34.234554,-118.800483,5.81
8270,2016,12,34.122593,-118.722263,6.09


In [101]:
fire_final = (
    fire_ready.merge(
        fire_pm25, on=["fire_lat", "fire_lon", "year", "month"], how="left"
    )
    .drop_duplicates()
    .reset_index(drop=True)
)
fire_final

,fire_lat_lon,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month,fire_pm25
0,"(35.25539508507167, -119.5812970773511)",238.031830,35.255395,-119.581297,0.032258,342.554565,1.595909,199305,1993,5,9.38
1,"(39.45165114943642, -120.0093608630314)",19.139362,39.451651,-120.009361,0.032258,0.700786,1.000503,199307,1993,7,5.53
2,"(38.63102591471577, -120.8402934705149)",70.179840,38.631026,-120.840293,0.033333,58.361294,0.271346,199309,1993,9,14.24
3,"(35.42095813876129, -119.7656494555765)",23.423927,35.420958,-119.765649,0.033333,331.157288,1.023493,199309,1993,9,11.99
4,"(35.32190356232154, -118.4169237147862)",151.780167,35.321904,-118.416924,0.032258,7.345666,1.831764,199308,1993,8,9.19
...,...,...,...,...,...,...,...,...,...,...,...
6632,"(37.87494989081461, -119.7281350256987)",331.624695,37.874950,-119.728135,0.166667,73.448456,0.113857,200709,2007,9,4.76
6633,"(37.56860595616687, -119.6600328204804)",23.633701,37.568606,-119.660033,0.233333,158.267761,0.253292,200711,2007,11,4.38
6634,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,1.000000,215.334213,0.138538,200711,2007,11,1.70
6635,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,0.258065,178.254211,0.474964,200712,2007,12,3.80


In [102]:
sorted(fire_final.year.unique())

[1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016]

In [103]:
def distance(wind, fire):
    """
    params:
    -------
    wind: wind cleaned data
    fire: fire semi-cleaned data

    return:
    -------
    df3: dataframe with distances from each fire during time period
    """
    # create labels
    wind["wind_lat_lon"] = [str(xy) for xy in zip(wind.zip_lat, wind.zip_lon)]
    fire["fire_lat_lon"] = [str(xy) for xy in zip(fire.fire_lat, fire.fire_lon)]

    ## for each point in wind data find the nearest point in the census data ##
    ###############
    # keep only unique points in fire data
    df1_unique = wind.drop_duplicates(["wind_lat_lon"])
    df2_unique = fire.drop_duplicates(["fire_lat_lon"])

    df1_unique.reset_index(drop=True, inplace=True)
    df2_unique.reset_index(drop=True, inplace=True)

    # transform to radians
    df1_unique["lat_r"] = np.radians(df1_unique.zip_lat)
    df1_unique["lon_r"] = np.radians(df1_unique.zip_lon)
    df2_unique["lat_r"] = np.radians(df2_unique.fire_lat)
    df2_unique["lon_r"] = np.radians(df2_unique.fire_lon)

    # compute pairwise distance (in miles)
    dist_matrix = (
        dist.pairwise(
            df2_unique[["lat_r", "lon_r"]],
            df1_unique[["lat_r", "lon_r"]],
        )
    ) * 3959

    # create a df from dist_matrix
    dist_matrix = pd.DataFrame(
        dist_matrix, index=df2_unique["fire_lat_lon"], columns=df1_unique["ZCTA"]
    )
    return dist_matrix

In [104]:
temp_df = distance(wind_final, fire_final)
dist_df = (
    fire_final.merge(
        temp_df, left_on="fire_lat_lon", right_on=temp_df.index, how="left"
    )
    .drop_duplicates()
    .reset_index(drop=True)
)
dist_df

,fire_lat_lon,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month,...,96140,96141,96142,96143,96145,96146,96148,96150,96161,97635
0,"(35.25539508507167, -119.5812970773511)",238.031830,35.255395,-119.581297,0.032258,342.554565,1.595909,199305,1993,5,...,273.902165,262.986709,255.763437,275.094422,265.225259,271.526768,276.316781,241.513929,275.412483,466.944962
1,"(39.45165114943642, -120.0093608630314)",19.139362,39.451651,-120.009361,0.032258,0.700786,1.000503,199307,1993,7,...,18.155774,30.531542,36.890962,15.868377,29.049318,24.497956,15.164144,49.742314,28.990727,176.052420
2,"(38.63102591471577, -120.8402934705149)",70.179840,38.631026,-120.840293,0.033333,58.361294,0.271346,199309,1993,9,...,55.923517,44.852443,42.502675,60.493373,45.091172,47.913347,59.138386,44.100498,44.201396,234.375215
3,"(35.42095813876129, -119.7656494555765)",23.423927,35.420958,-119.765649,0.033333,331.157288,1.023493,199309,1993,9,...,261.622226,250.503922,243.318349,262.985454,252.701931,258.958683,264.112075,229.258884,262.543176,454.860185
4,"(35.32190356232154, -118.4169237147862)",151.780167,35.321904,-118.416924,0.032258,7.345666,1.831764,199308,1993,8,...,283.319934,274.116537,266.888233,283.351792,276.555557,282.960181,285.162251,251.917314,288.832200,471.414260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6632,"(37.87494989081461, -119.7281350256987)",331.624695,37.874950,-119.728135,0.166667,73.448456,0.113857,200709,2007,9,...,93.579473,83.601955,76.330517,94.315845,85.999136,92.411551,95.759703,61.483932,98.022876,285.885768
6633,"(37.56860595616687, -119.6600328204804)",23.633701,37.568606,-119.660033,0.233333,158.267761,0.253292,200711,2007,11,...,115.058673,104.965884,97.687268,115.806581,107.344223,113.752581,117.250544,82.902744,119.099573,307.313468
6634,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,1.000000,215.334213,0.138538,200711,2007,11,...,102.401609,92.355699,85.079677,103.152564,94.742784,101.153523,104.591706,70.267223,106.628801,294.707998
6635,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,0.258065,178.254211,0.474964,200712,2007,12,...,102.401609,92.355699,85.079677,103.152564,94.742784,101.153523,104.591706,70.267223,106.628801,294.707998


In [105]:
sorted(dist_df.year_month.unique())

[199101,
 199102,
 199104,
 199106,
 199107,
 199108,
 199109,
 199110,
 199111,
 199112,
 199202,
 199203,
 199205,
 199206,
 199207,
 199208,
 199209,
 199210,
 199211,
 199305,
 199306,
 199307,
 199308,
 199309,
 199310,
 199311,
 199312,
 199401,
 199405,
 199406,
 199407,
 199408,
 199409,
 199410,
 199412,
 199505,
 199506,
 199507,
 199508,
 199509,
 199510,
 199511,
 199512,
 199601,
 199604,
 199605,
 199606,
 199607,
 199608,
 199609,
 199610,
 199611,
 199703,
 199704,
 199705,
 199706,
 199707,
 199708,
 199709,
 199710,
 199804,
 199806,
 199807,
 199808,
 199809,
 199810,
 199901,
 199902,
 199903,
 199904,
 199905,
 199906,
 199907,
 199908,
 199909,
 199910,
 199911,
 199912,
 200001,
 200003,
 200004,
 200005,
 200006,
 200007,
 200008,
 200009,
 200010,
 200011,
 200101,
 200103,
 200105,
 200106,
 200107,
 200108,
 200109,
 200110,
 200111,
 200112,
 200201,
 200202,
 200203,
 200204,
 200205,
 200206,
 200207,
 200208,
 200209,
 200210,
 200211,
 200212,
 200301,
 

In [106]:
def fire_bearings(wind, fire):
    """
    params:
    -------
    wind: wind cleaned data
    fire: fire semi-cleaned data

    return:
    -------
    df3: dataframe with distances from each fire during time period
    """

    ## for each point in wind data find the angle##
    ###############
    # keep only unique points in fire data
    df1_unique = wind.drop_duplicates(["wind_lat_lon"])
    df2_unique = fire.drop_duplicates(["fire_lat_lon"])

    df1_unique.reset_index(drop=True, inplace=True)
    df2_unique.reset_index(drop=True, inplace=True)

    # transform to radians
    df1_unique["lat_r"] = np.radians(df1_unique.zip_lat)
    df1_unique["lon_r"] = np.radians(df1_unique.zip_lon)
    df2_unique["lat_r"] = np.radians(df2_unique.fire_lat)
    df2_unique["lon_r"] = np.radians(df2_unique.fire_lon)

    # compute pairwise distance (in miles)
    bearing_matrix = np.zeros((df1_unique.shape[0], df2_unique.shape[0]))
    for idx in range(df1_unique.shape[0]):
        pa_lat = df1_unique["zip_lat"][idx]
        pa_lon = df1_unique["zip_lon"][idx]
        wdir_ba = df2_unique["fire_wdir"][idx]
        for idy in range(df2_unique.shape[0]):
            pb_lat = df2_unique["fire_lat"][idy]
            pb_lon = df2_unique["fire_lon"][idy]

            if pb_lon > pa_lon:
                d_X = -np.cos(pa_lat) * np.sin(pa_lon - pb_lon)
            else:
                d_X = np.cos(pa_lat) * np.sin(pa_lon - pb_lon)

            if pb_lat > pa_lat:
                d_Y = -np.cos(pb_lat) * np.sin(pa_lat) - np.sin(pb_lat) * np.cos(
                    pa_lat
                ) * np.cos(pa_lon - pb_lon)
            else:
                d_Y = np.cos(pb_lat) * np.sin(pa_lat) - np.sin(pb_lat) * np.cos(
                    pa_lat
                ) * np.cos(pa_lon - pb_lon)

            B_ba = np.arctan2(d_X, d_Y) * (180 / np.pi) % 360

            angle_diff = np.max([B_ba, wdir_ba]) - np.min([B_ba, wdir_ba])

            if angle_diff > 180:
                upwind_effect = (angle_diff) / 360
            else:
                upwind_effect = 1 - (angle_diff) / 360

            bearing_matrix[idx, idy] = upwind_effect

    # create a df from dist_matrix
    bearing_matrix = pd.DataFrame(
        bearing_matrix, index=df1_unique["ZCTA"], columns=df2_unique["fire_lat_lon"]
    )

    return bearing_matrix

In [107]:
temp_df = fire_bearings(wind_final, fire_final)

KeyboardInterrupt: 

In [ ]:
temp_df = temp_df.fillna(0)
merge_temp_df = temp_df.T
bearings = fire_ready.merge(
    merge_temp_df, left_on="fire_lat_lon", right_on=merge_temp_df.index, how="left"
)
bearings_fire = bearings

In [ ]:
bearings_fire

In [ ]:
# treatment_df = (bearings_fire.drop_duplicates().iloc[:, 11:] >= 0.75)[
#     dist_df.drop_duplicates().iloc[:, 11:] < 200
# ].fillna(False)

# treatment_df["fire_lat_lon"] = bearings.drop_duplicates()["fire_lat_lon"].astype(str)
# treatment_df["year_month"] = bearings_fire.drop_duplicates()["year_month"]
# treatment_df["fire_wdir"] = bearings_fire.drop_duplicates()["fire_wdir"]
# treatment_df["fire_wspd"] = bearings_fire.drop_duplicates()["fire_wspd"]

# treatment_df["fire_days_in_mo"] = bearings_fire.drop_duplicates()["fire_days_in_mo"]
# treatment_df = treatment_df.reset_index(drop=True)

In [ ]:
fire_bear = bearings_fire
# [
#     (bearings_fire.drop_duplicates().iloc[:, 11:] >= 0.75)[
#         dist_df.drop_duplicates().iloc[:, 11:] < 200
#     ]
# ].fillna(0)

fire_bear["fire_lat_lon"] = bearings.drop_duplicates()["fire_lat_lon"].astype(str)
fire_bear["year_month"] = bearings_fire.drop_duplicates()["year_month"]
fire_bear["fire_wdir"] = bearings_fire.drop_duplicates()["fire_wdir"]
fire_bear["fire_wspd"] = bearings_fire.drop_duplicates()["fire_wspd"]

fire_bear["fire_days_in_mo"] = bearings_fire.drop_duplicates()["fire_days_in_mo"]
fire_bear = fire_bear.reset_index(drop=True)

# fire_bear.to_csv("fire_bear.csv")

In [ ]:
fire_dist = dist_df
# [
#     (bearings_fire.drop_duplicates().iloc[:, 11:] >= 0.75)[
#         dist_df.drop_duplicates().iloc[:, 11:] < 200
#     ]
# ].fillna(0)

fire_dist["fire_lat_lon"] = bearings.drop_duplicates()["fire_lat_lon"].astype(str)
fire_dist["year_month"] = bearings_fire.drop_duplicates()["year_month"]
fire_dist["fire_wdir"] = bearings_fire.drop_duplicates()["fire_wdir"]
fire_dist["fire_wspd"] = bearings_fire.drop_duplicates()["fire_wspd"]

fire_dist["fire_days_in_mo"] = bearings_fire.drop_duplicates()["fire_days_in_mo"]
fire_dist = fire_dist.reset_index(drop=True)

# fire_dist.to_csv("fire_dist.csv")

In [ ]:
days_mo = fire_dist["fire_days_in_mo"][2]
wspd = fire_dist["fire_wspd"][2]
distance = fire_dist["96142"][2]
bearing = fire_bear["96142"][2]

instrument = np.sum(days_mo * (wspd / distance) * bearing)
instrument

NameError: ignored

In [ ]:
fire_dist = pd.read_csv(in_instrument + "fire_dist.csv", index_col=1)
fire_bear = pd.read_csv(in_instrument + "fire_bear.csv", index_col=1)

In [ ]:
import multiprocessing

from tqdm import tqdm

iters = int(np.ceil(fire_dist.shape[0] / 100))
instrument_df = pd.DataFrame(
    columns=[
        "ZCTA",
        "fire_lat_lon",
        "instrument",
        "instrument_norm",
        "year_month",
        "bearing",
        "distance",
        "fire_wspd",
    ]
)

In [ ]:
fire_final

,fire_lat_lon,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month,fire_pm25
0,"(35.25539508507167, -119.5812970773511)",238.031830,35.255395,-119.581297,0.032258,342.554565,1.595909,199305,1993,5,9.38
1,"(39.45165114943642, -120.0093608630314)",19.139362,39.451651,-120.009361,0.032258,0.700786,1.000503,199307,1993,7,5.53
2,"(38.63102591471577, -120.8402934705149)",70.179840,38.631026,-120.840293,0.033333,58.361294,0.271346,199309,1993,9,14.24
3,"(35.42095813876129, -119.7656494555765)",23.423927,35.420958,-119.765649,0.033333,331.157288,1.023493,199309,1993,9,11.99
4,"(35.32190356232154, -118.4169237147862)",151.780167,35.321904,-118.416924,0.032258,7.345666,1.831764,199308,1993,8,9.19
...,...,...,...,...,...,...,...,...,...,...,...
6632,"(37.87494989081461, -119.7281350256987)",331.624695,37.874950,-119.728135,0.166667,73.448456,0.113857,200709,2007,9,4.76
6633,"(37.56860595616687, -119.6600328204804)",23.633701,37.568606,-119.660033,0.233333,158.267761,0.253292,200711,2007,11,4.38
6634,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,1.000000,215.334213,0.138538,200711,2007,11,1.70
6635,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,0.258065,178.254211,0.474964,200712,2007,12,3.80


In [ ]:
instrument_df

,ZCTA,fire_lat_lon,instrument,instrument_norm,year_month,bearing,distance,fire_wspd
0,95415,"(41.67609857663389, -123.4192304813263)",0.000621,0.017956,200709,0.952293,189.911753,0.123880
1,95417,"(41.67609857663389, -123.4192304813263)",0.000829,0.025135,200709,0.965177,144.313825,0.123880
2,95420,"(41.67609857663389, -123.4192304813263)",0.000760,0.025194,200709,0.991477,161.572997,0.123880
3,95422,"(41.67609857663389, -123.4192304813263)",0.000601,0.017003,200709,0.946955,195.269309,0.123880
4,95423,"(41.67609857663389, -123.4192304813263)",0.000614,0.017552,200709,0.949258,191.370069,0.123880
...,...,...,...,...,...,...,...,...
11478,96145,"(37.57265198276752, -119.6631369569977)",0.000047,0.001219,200712,0.925093,107.028721,0.167112
11479,96146,"(37.57265198276752, -119.6631369569977)",0.000042,0.000921,200712,0.890247,113.436889,0.167112
11480,96148,"(37.57265198276752, -119.6631369569977)",0.000041,0.000883,200712,0.888689,116.944580,0.167112
11481,96150,"(37.57265198276752, -119.6631369569977)",0.000056,0.000982,200712,0.858914,82.590346,0.167112


In [ ]:
fire_dist.fire_wspd.dtype

dtype('float64')

In [ ]:
list(fire_dist.columns).index("fire_lat_lon")

0

In [ ]:
def get_instrument(distance_df, bearing_df, instrument_df):
    # dist_max = np.max(np.max(distance_df.iloc[:, 11:]))
    # dist_min = 0
    # wspd_max = np.max(distance_df["fire_wspd"])
    # wspd_min = 0
    # acres_max = np.max(distance_df["acres"])
    # acres_min = np.min(distance_df["acres"])

    for ym in tqdm(range(distance_df.shape[0]), leave=False):
        days_mo = distance_df.iloc[ym, 24]
        wspd = distance_df.iloc[ym, 6]  # ["fire_wspd"][ym]
        for zcta in tqdm(range(11, distance_df.shape[1] - 1), leave=False):
            distance = distance_df.iloc[ym, zcta]
            bearing = bearing_df.iloc[ym, zcta]
            # instrument = days_mo * wspd * bearing / distance
            # dist_norm = (distance) / (dist_max)
            # wspd_norm = (wspd) / (wspd_max)
            # bear_norm = (bearing - 0.5) / (0.5)
            # instrument_norm = days_mo * wspd_norm * bear_norm / dist_norm

            instrument_df = instrument_df.append(
                {
                    "ZCTA": zcta,
                    "fire_lat_lon": distance_df.iloc[ym, 0],
                    "instrument": np.nan,
                    "year_month": distance_df.iloc[ym, 7],  # ["year_month"][ym],
                    "bearing": bearing,
                    "dist": distance,
                    "fire_wspd": wspd,
                    # "new_bearing": bear_norm,
                    # "wspd_scaled": wspd_norm,
                    # "dist_scaled": dist_norm,
                    # "instrument_norm": instrument_norm,
                },
                ignore_index=True,
            )
    return instrument_df

In [ ]:
fire_dist.shape[0]

6637

In [ ]:
def get_instrument_multi(iter_1, iter_2):
    instrument_df = pd.DataFrame(
        columns=[
            "ZCTA",
            "fire_lat_lon",
            "instrument",
            "year_month",
            "bearing",
            "new_bearing",
            "dist",
            # "instrument_norm",
            # "fire_wspd",
            # "wspd_scaled",
            # "dist_scaled",
        ]
    )

    distance_df = fire_dist.iloc[iter_1:iter_2, :]
    bearing_df = fire_bear.iloc[iter_1:iter_2, :]

    dist_max = np.max(np.max(distance_df.iloc[:, 11:]))
    dist_min = 0
    wspd_max = np.max(distance_df["fire_wspd"])
    wspd_min = 0
    acres_max = np.max(distance_df["acres"])
    acres_min = np.min(distance_df["acres"])

    for ym in tqdm(distance_df.index):
        days_mo = distance_df["fire_days_in_mo"][ym]
        wspd = distance_df["fire_wspd"][ym]
        for zcta in distance_df.columns[11:]:
            distance = distance_df[zcta][ym]
            bearing = bearing_df[zcta][ym]
            instrument = days_mo * wspd * bearing / distance

            #             dist_norm = (distance) / (dist_max)
            #             wspd_norm = (wspd) / (wspd_max)
            #             bear_norm = (bearing - 0.5) / (0.5)
            #             instrument_norm = days_mo * wspd_norm * bear_norm / dist_norm

            instrument_df = instrument_df.append(
                {
                    "ZCTA": zcta,
                    "fire_lat_lon": distance_df["fire_lat_lon"][ym],
                    "instrument": instrument,
                    "year_month": distance_df["year_month"][ym],
                    "bearing": bearing,
                    "dist": distance,
                    "fire_wspd": wspd,
                    # "new_bearing": bear_norm,
                    # "wspd_scaled": wspd_norm,
                    # "dist_scaled": dist_norm,
                    # "instrument_norm": instrument_norm,
                },
                ignore_index=True,
            )
    pbar.update(1)
    print(f"process {iteration} complete")
    instrument_df.to_csv(in_instrument + f"multi-{iter_1}-{iter_2}.csv")
    return f"{iter_1}-{iter_2}"

In [ ]:
from modules.instrument_multi import get_instrument_multi as gim

In [ ]:
%cd notebooks

[Errno 2] No such file or directory: 'notebooks'
/Users/jordan/Documents/GitHub/fire_capstone/notebooks


In [ ]:
SIZE = 50
START = 0

iters = int(np.ceil(fire_dist.shape[0] / SIZE))
items = params = [(i * SIZE, i * SIZE + SIZE) for i in range(START, iters)]
tasks = range(iters)

In [ ]:
# import multiprocessing as mp

# # Step 1: Use multiprocessing.Pool() and specify number of cores to use (here I use 4).
# pool = mp.Pool(2)
# pbar = tqdm(total=len(tasks))

# with mp.Pool(4) as pool:
#     # Step 2: Use pool.starmap which takes a multiple iterable arguments
#     pool.starmap(gim, items)
#     # print(f'Got result: {result}', flush=True)

# pbar.close()

In [ ]:
SIZE = 50
START = 43

iters = int(np.ceil(fire_dist.shape[0] / SIZE))
items = params = [(i * SIZE, i * SIZE + SIZE) for i in range(START, iters)]
tasks = range(iters)

for i in tqdm(range(START, iters)):
    instrument_df = pd.DataFrame(
        columns=[
            "ZCTA",
            "fire_lat_lon",
            "instrument",
            "year_month",
            "bearing",
            "dist",
            "fire_wspd",
            # "wspd_scaled",
            # "dist_scaled",
            # "instrument_norm",
            # "new_bearing",
        ]
    )

    instrument_df = get_instrument(
        fire_dist[i * SIZE : i * SIZE + SIZE],
        fire_bear[i * SIZE : i * SIZE + SIZE],
        instrument_df,
    )
    instrument_df.to_csv(in_instrument + f"iter_{i}.csv")

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

...

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

In [ ]:
print("woo")

woo


## Consolidate iterations of lookups and merge for final df

In [67]:
import glob

files = glob.glob(in_instrument + "iter" + "*.csv")
df_all_instruments = pd.DataFrame()

total = 0
for f in files:
    total += 1
    csv = pd.read_csv(f, index_col=0)
    df_all_instruments = df_all_instruments.append(csv)
print(f"{total} files consoidated.")

df_all_instruments = df_all_instruments.reset_index(drop=True).drop_duplicates()
df_all_instruments.to_csv(in_instrument + "non_agg_instrument.csv")

133 files consoidated.


In [2]:
# instruments = df_all_instruments.reset_index(drop=True)


# instrument_final = (
#     instruments.groupby(["ZCTA", "year_month"]).aggregate(sum).drop_duplicates()
# )
# instrument_final.to_csv(in_instrument + "instrument_final.csv")

In [110]:
df_all_instruments= pd.read_csv(in_instrument + "non_agg_instrument.csv")

In [3]:
# instruments = df_all_instruments.reset_index(drop=True)

In [111]:
wind_elevation = pd.read_csv("all_elevations.csv")
wind_elevation["zips"] = wind_elevation["zips"].astype(str)
wind_elevation["elevation"] = (
    wind_elevation["elevation"].map(lambda x: x[:-2]).astype(float)
)
wind_elevation = wind_final.merge(
    wind_elevation, left_on="ZCTA", right_on="zips", how="left"
)
wind_elevation = wind_elevation.rename(columns={"elevation": "zip_elevation"})

In [112]:
fire_elevation = pd.read_csv(in_instrument + "elevation_final.csv", index_col=0)
fire_elevation = fire_final.merge(
    fire_elevation, on=["fire_lat", "fire_lon"], how="left"
)

In [113]:
fire_elevation["year_month"] = fire_elevation["year_month"].astype(str)
df_all_instruments["ZCTA"] = df_all_instruments["ZCTA"].astype(str)
wind_elevation["ZCTA"] = wind_elevation["ZCTA"].astype(str)
df_all_instruments["year_month"] = df_all_instruments["year_month"].astype(str)

In [114]:
temp_instrument = df_all_instruments.drop_duplicates().merge(
    fire_elevation, on=["fire_lat_lon", "year_month"], how="left"
)

In [132]:
len(temp_instrument.year.unique())

26

In [131]:
len(temp_fire_df.year.unique())

26

In [133]:
temp_wind_df = wind_elevation[
    ["ZCTA", "zip_elevation", "zip_pm25", "year_month"]
].drop_duplicates()

In [145]:
temp_wind_df['year']=temp_wind_df['year_month'].apply(lambda x: str(x)[:4])
temp_wind_df['month']=temp_wind_df['year_month'].apply(lambda x: str(x)[-2:])

In [146]:
temp_instrument.columns

Index(['Unnamed: 0', 'ZCTA', 'fire_lat_lon', 'instrument', 'year_month',
       'bearing', 'dist', 'fire_wspd_x', 'acres', 'fire_lat', 'fire_lon',
       'fire_days_in_mo', 'fire_wdir', 'fire_wspd_y', 'year', 'month',
       'fire_pm25', 'fire_elevation'],
      dtype='object')

In [147]:
temp_fire_df = temp_instrument.rename(columns={"fire_wspd_x": "fire_wspd"})

In [148]:
temp_temp = temp_wind_df.merge(
    temp_fire_df, on=["ZCTA", "year_month"], how ='left'
).drop_duplicates()

In [149]:
temp_temp["elevation_difference"] = (
    temp_temp["fire_elevation"] - temp_temp["zip_elevation"]
)

In [156]:
temp_temp.columns

Index(['ZCTA', 'zip_elevation', 'zip_pm25', 'year_month', 'year_x', 'month_x',
       'Unnamed: 0', 'fire_lat_lon', 'instrument', 'bearing', 'dist',
       'fire_wspd', 'acres', 'fire_lat', 'fire_lon', 'fire_days_in_mo',
       'fire_wdir', 'fire_wspd_y', 'year_y', 'month_y', 'fire_pm25',
       'fire_elevation', 'elevation_difference'],
      dtype='object')

In [159]:
last_temp = temp_temp[['ZCTA', 'zip_elevation', 'zip_pm25', 'year_month', 'year_x', 'month_x', 'fire_lat_lon', 'instrument', 'bearing', 'dist',
       'fire_wspd', 'acres', 'fire_lat', 'fire_lon', 'fire_days_in_mo',
       'fire_wdir','fire_pm25',
       'fire_elevation', 'elevation_difference']]
temp_temp = last_temp.rename(columns={'year_x':'year','month_x':'month'})           

In [123]:
wind_ready.merge

,lat,lon,ZCTA,u,v,wdir,wspd,year_month,year,month
0,37.465,-117.936,89010,0.504258,-0.719008,305.042938,0.878208,199101,1991,1
1,35.396,-116.322,89019,-0.172753,-0.946940,259.661102,0.962568,199101,1991,1
2,36.161,-116.139,89060,-0.435964,-0.812957,241.796738,0.922477,199101,1991,1
3,35.957,-115.897,89061,-0.560538,-1.176384,244.522552,1.303105,199101,1991,1
4,39.520,-120.032,89439,0.042253,0.205661,78.390099,0.209957,199101,1991,1
...,...,...,...,...,...,...,...,...,...,...
627067,39.149,-120.248,96146,0.636599,0.953782,56.279018,1.146716,202212,2022,12
627068,39.236,-120.062,96148,0.636599,0.953782,56.279018,1.146716,202212,2022,12
627069,38.732,-120.033,96150,0.071517,0.472164,81.387100,0.477549,202212,2022,12
627070,39.184,-120.427,96161,-0.217010,0.239125,132.224274,0.322915,202212,2022,12


## Adding normalization (min-max) for wspd, distance, acres, bearing

In [160]:
temp_max = np.max(temp_temp["fire_wspd"])
temp_min = np.min(temp_temp["fire_wspd"])
diff = temp_max - temp_min

temp_temp["fire_wspd_norm"] = temp_temp["fire_wspd"].map(
    lambda x: ((x - temp_min) / (diff))
)
print("Windspeed normalized")

temp_max = np.max(temp_temp["dist"])
temp_min = np.min(temp_temp["dist"])
diff = temp_max - temp_min

temp_temp["dist_norm"] = temp_temp["dist"].map(lambda x: ((x - temp_min) / (diff)))
print("Distance normalized")

temp_max = np.max(temp_temp["acres"])
temp_min = np.min(temp_temp["acres"])
diff = temp_max - temp_min

temp_temp["acres_norm"] = temp_temp["acres"].map(lambda x: ((x - temp_min) / (diff)))
print("Acres normalized")

temp_max = 1
temp_min = 0.5
diff = temp_max - temp_min

temp_temp["bearing_norm"] = temp_temp["bearing"].map(
    lambda x: ((x - temp_min) / (diff))
)
print("Bearings normalized")

Windspeed normalized
Distance normalized
Acres normalized
Bearings normalized


In [161]:
temp_temp["bearing_bins"] = temp_temp["bearing"].map(
    lambda x: 10
    if x > 0.95
    else (
        9
        if x > 0.9
        else (
            8
            if x > 0.85
            else (
                7
                if x > 0.8
                else (
                    6
                    if x > 0.75
                    else (
                        5
                        if x > 0.7
                        else (
                            4
                            if x > 0.65
                            else (3 if x > 0.6 else (2 if x > 0.55 else (1)))
                        )
                    )
                )
            )
        )
    )
)

In [162]:
temp_temp.drop_duplicates()

,ZCTA,zip_elevation,zip_pm25,year_month,year,month,fire_lat_lon,instrument,bearing,dist,...,fire_days_in_mo,fire_wdir,fire_pm25,fire_elevation,elevation_difference,fire_wspd_norm,dist_norm,acres_norm,bearing_norm,bearing_bins
0,89010,4784.0,12.450976,199101,1991,01,"(39.15535115700015, -122.7759650805785)",0.000163,0.782605,287.182949,...,0.064516,274.314545,0.10,1320.0,-3464.0,0.174642,0.349547,0.000300,0.565210,6
1,89010,4784.0,12.450976,199101,1991,01,"(39.56756030998135, -121.0023355975935)",0.000071,0.938027,220.403951,...,0.032258,186.630615,11.24,1263.0,-3521.0,0.096410,0.268253,0.000117,0.876054,9
2,89010,4784.0,12.450976,199101,1991,01,"(39.08499688682003, -122.7444818242734)",0.000167,0.790398,283.790088,...,0.064516,274.314545,1.74,1013.0,-3771.0,0.174642,0.345417,0.000328,0.580796,6
3,89010,4784.0,12.450976,199101,1991,01,"(39.75680610240665, -121.3315226354367)",0.000055,0.996625,242.222915,...,0.096774,182.438004,10.59,1057.0,-3727.0,0.024829,0.294814,0.000211,0.993251,10
4,89010,4784.0,12.450976,199101,1991,01,"(39.69440333991243, -121.355888160909)",0.000207,0.994434,240.497549,...,0.096774,186.630615,15.02,852.0,-3932.0,0.096410,0.292714,0.000561,0.988867,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5692633,96146,6649.0,4.626316,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5692634,96148,6473.0,4.528572,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5692635,96150,6548.0,4.286667,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5692636,96161,6042.0,5.053509,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [163]:
temp_temp["ins_1_no_bin_raw"] = (
    temp_temp["bearing"] * temp_temp["fire_wspd"] / temp_temp["dist"]
)
temp_temp["ins_2_add_acres_raw"] = (
    temp_temp["bearing"]
    * temp_temp["fire_wspd"]
    * temp_temp["acres"]
    / temp_temp["dist"]
)
temp_temp["ins_3_norms"] = (
    temp_temp["bearing_norm"] * temp_temp["fire_wspd_norm"] / temp_temp["dist_norm"]
)
temp_temp["ins_4_add_acres_norms"] = (
    temp_temp["bearing_norm"]
    * temp_temp["fire_wspd_norm"]
    * temp_temp["acres_norm"]
    / temp_temp["dist_norm"]
)
temp_temp["ins_5_norm_dist_numerator"] = (
    temp_temp["bearing_norm"]
    * temp_temp["fire_wspd_norm"]
    * temp_temp["acres_norm"]
    / temp_temp["dist_norm"]
)
temp_temp["ins_6_sub_bins_raw"] = (
    temp_temp["bearing_bins"]
    * temp_temp["fire_wspd"]
    * temp_temp["acres"]
    * temp_temp["dist"]
)

In [165]:
temp_temp.drop_duplicates()

,ZCTA,zip_elevation,zip_pm25,year_month,year,month,fire_lat_lon,instrument,bearing,dist,...,dist_norm,acres_norm,bearing_norm,bearing_bins,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_dist_numerator,ins_6_sub_bins_raw
0,89010,4784.0,12.450976,199101,1991,01,"(39.15535115700015, -122.7759650805785)",0.000163,0.782605,287.182949,...,0.349547,0.000300,0.565210,6,0.002533,0.113662,0.282393,0.000085,0.000085,71868.672976
1,89010,4784.0,12.450976,199101,1991,01,"(39.56756030998135, -121.0023355975935)",0.000071,0.938027,220.403951,...,0.268253,0.000117,0.876054,9,0.002197,0.038422,0.314854,0.000037,0.000037,17908.026722
2,89010,4784.0,12.450976,199101,1991,01,"(39.08499688682003, -122.7444818242734)",0.000167,0.790398,283.790088,...,0.345417,0.000328,0.580796,6,0.002588,0.126796,0.293650,0.000096,0.000096,77518.253168
3,89010,4784.0,12.450976,199101,1991,01,"(39.75680610240665, -121.3315226354367)",0.000055,0.996625,242.222915,...,0.294814,0.000211,0.993251,10,0.000568,0.017893,0.083651,0.000018,0.000018,10533.561275
4,89010,4784.0,12.450976,199101,1991,01,"(39.69440333991243, -121.355888160909)",0.000207,0.994434,240.497549,...,0.292714,0.000561,0.988867,10,0.002134,0.179142,0.325700,0.000183,0.000183,104193.925266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5692633,96146,6649.0,4.626316,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
5692634,96148,6473.0,4.528572,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
5692635,96150,6548.0,4.286667,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
5692636,96161,6042.0,5.053509,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
temp_temp.to_csv(in_instrument + "new_non_agg.csv")

## Aggregation of the dataframe for OLS

In [167]:
instrument_df = pd.read_csv(in_instrument + "new_non_agg.csv")
instrument_df

,Unnamed: 0,ZCTA,zip_elevation,zip_pm25,year_month,year,month,fire_lat_lon,instrument,bearing,...,dist_norm,acres_norm,bearing_norm,bearing_bins,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_dist_numerator,ins_6_sub_bins_raw
0,0,89010,4784.0,12.450976,199101,1991,1,"(39.15535115700015, -122.7759650805785)",0.000163,0.782605,...,0.349547,0.000300,0.565210,6,0.002533,0.113662,0.282393,0.000085,0.000085,71868.672976
1,1,89010,4784.0,12.450976,199101,1991,1,"(39.56756030998135, -121.0023355975935)",0.000071,0.938027,...,0.268253,0.000117,0.876054,9,0.002197,0.038422,0.314854,0.000037,0.000037,17908.026722
2,2,89010,4784.0,12.450976,199101,1991,1,"(39.08499688682003, -122.7444818242734)",0.000167,0.790398,...,0.345417,0.000328,0.580796,6,0.002588,0.126796,0.293650,0.000096,0.000096,77518.253168
3,3,89010,4784.0,12.450976,199101,1991,1,"(39.75680610240665, -121.3315226354367)",0.000055,0.996625,...,0.294814,0.000211,0.993251,10,0.000568,0.017893,0.083651,0.000018,0.000018,10533.561275
4,4,89010,4784.0,12.450976,199101,1991,1,"(39.69440333991243, -121.355888160909)",0.000207,0.994434,...,0.292714,0.000561,0.988867,10,0.002134,0.179142,0.325700,0.000183,0.000183,104193.925266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3765693,5692633,96146,6649.0,4.626316,201812,2018,12,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
3765694,5692634,96148,6473.0,4.528572,201812,2018,12,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
3765695,5692635,96150,6548.0,4.286667,201812,2018,12,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
3765696,5692636,96161,6042.0,5.053509,201812,2018,12,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
instrument_df.year

# START HERE FOR AGGREGATION

In [169]:
temp_temp = pd.read_csv(in_instrument + "new_non_agg.csv")

In [170]:
temp_temp.year.unique()

array([1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018])

In [171]:
wind_ready = pd.read_csv(in_dir + "wind_zip_pm25.csv")
wind_ready

,Unnamed: 0,zip_lat,zip_lon,ZCTA,zip_wdir,zip_wspd,year_month,year,month,zip_pm25
0,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976
1,1,35.396,-116.322,89019,259.661102,0.962568,199101,1991,1,10.846541
2,2,36.161,-116.139,89060,241.796738,0.922477,199101,1991,1,12.385000
3,3,35.957,-115.897,89061,244.522552,1.303105,199101,1991,1,10.150000
4,4,39.520,-120.032,89439,78.390099,0.209957,199101,1991,1,8.389565
...,...,...,...,...,...,...,...,...,...,...
548683,548683,39.149,-120.248,96146,76.585587,0.437017,201812,2018,12,4.626316
548684,548684,39.236,-120.062,96148,76.585587,0.437017,201812,2018,12,4.528572
548685,548685,38.732,-120.033,96150,107.369316,0.231856,201812,2018,12,4.286667
548686,548686,39.184,-120.427,96161,151.071350,0.243283,201812,2018,12,5.053509


In [172]:
#Revision 3
temp_temp["ins_1_no_bin_raw"] = (
    temp_temp["bearing_norm"] 
    * temp_temp["fire_wspd"] 
    / temp_temp["dist"]
)
temp_temp["ins_2_add_acres_raw"] = (
    temp_temp["bearing_norm"]
    * temp_temp["fire_wspd"]
    * temp_temp["acres"]
    / temp_temp["dist"]
)
temp_temp["ins_3_norms"] = (
    temp_temp["bearing_norm"] 
    * temp_temp["fire_wspd_norm"] 
    / temp_temp["dist_norm"]
)
temp_temp["ins_4_add_acres_norms"] = (
    temp_temp["bearing_norm"]
    * temp_temp["fire_wspd_norm"]
    * temp_temp["acres_norm"]
    / temp_temp["dist_norm"]
)
temp_temp["ins_5_norm_bins_acres"] = (
    temp_temp["bearing_bins"]
    * temp_temp["fire_wspd_norm"]
    * temp_temp["acres_norm"]
    / temp_temp["dist_norm"]
)
temp_temp["ins_6_bins_raw"] = (
    temp_temp["bearing_bins"]
    * temp_temp["fire_wspd"]
    * temp_temp["acres"]
    / temp_temp["dist"]
)

In [173]:
temp_temp["year_month"] = temp_temp["year_month"].astype(str)
temp_temp["ZCTA"] = temp_temp["ZCTA"].astype(str)
wind_ready["year_month"] = wind_ready["year_month"].astype(str)
wind_ready["ZCTA"] = wind_ready["ZCTA"].astype(str)


In [174]:
instrument_df = wind_ready.merge(temp_temp, on= ['ZCTA','year_month'],how='left').rename(columns={'wspd':'zip_wspd',
                                                                                                 'zip_pm25_y':
                                                                                                'zip_pm25'})
instrument_df

,Unnamed: 0_x,zip_lat,zip_lon,ZCTA,zip_wdir,zip_wspd,year_month,year_x,month_x,zip_pm25_x,...,bearing_norm,bearing_bins,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_dist_numerator,ins_6_sub_bins_raw,ins_5_norm_bins_acres,ins_6_bins_raw
0,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976,...,0.565210,6,0.001829,0.082088,0.282393,0.000085,0.000085,71868.672976,0.000900,0.871410
1,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976,...,0.876054,9,0.002052,0.035884,0.314854,0.000037,0.000037,17908.026722,0.000378,0.368646
2,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976,...,0.580796,6,0.001902,0.093171,0.293650,0.000096,0.000096,77518.253168,0.000994,0.962519
3,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976,...,0.993251,10,0.000566,0.017832,0.083651,0.000018,0.000018,10533.561275,0.000177,0.179533
4,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976,...,0.988867,10,0.002122,0.178139,0.325700,0.000183,0.000183,104193.925266,0.001849,1.801445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3765693,548683,39.149,-120.248,96146,76.585587,0.437017,201812,2018,12,4.626316,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3765694,548684,39.236,-120.062,96148,76.585587,0.437017,201812,2018,12,4.528572,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3765695,548685,38.732,-120.033,96150,107.369316,0.231856,201812,2018,12,4.286667,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3765696,548686,39.184,-120.427,96161,151.071350,0.243283,201812,2018,12,5.053509,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
agg_dict = {
    "ins_1_no_bin_raw": "sum",
    "ins_2_add_acres_raw": "sum",
    "ins_3_norms": "sum",
    "ins_4_add_acres_norms": "sum",
    "ins_5_norm_bins_acres": "sum",
    "ins_6_bins_raw": "sum",
    "zip_pm25": "first",
    "zip_elevation": "mean",
    "elevation_difference": "mean",
    "zip_wspd":"first",
}

In [176]:
model_df2 = (
    instrument_df.groupby(  # [['ZCTA','year_month','fire_lat_lon','instrument','instrument_2','instrument_2_norm']]
        ["ZCTA", "year_month"]
    )
    .aggregate(agg_dict)
    .drop_duplicates()
)

In [177]:
model_df2

ins_1_no_bin_raw  ins_2_add_acres_raw  ins_3_norms  \
ZCTA  year_month                                                       
89010 199101              0.014313             2.171619     2.193307   
      199102              0.003401             1.473353     0.522705   
      199103              0.000000             0.000000     0.000000   
      199104              0.024555             4.317860     3.803710   
      199105              0.000000             0.000000     0.000000   
...                            ...                  ...          ...   
97635 201808              0.000000             0.000000     0.000000   
      201809              0.000000             0.000000     0.000000   
      201810              0.000000             0.000000     0.000000   
      201811              0.000000             0.000000     0.000000   
      201812              0.000000             0.000000     0.000000   

                  ins_4_add_acres_norms  ins_5_norm_bins_acres  \
ZCTA  year_month                                                 
89010 199101                   0.002237               0.024999   
      199102                   0.001519               0.017402   
      199103                   0.000000               0.000000   
      199104                   0.004475               0.053075   
      199105                   0.000000               0.000000   
...                                 ...                    ...   
97635 201808                   0.000000               0.000000   
      201809                   0.000000               0.000000   
      201810                   0.000000               0.000000   
      201811                   0.000000               0.000000   
      201812                   0.000000               0.000000   

                  ins_6_bins_raw   zip_pm25  zip_elevation  \
ZCTA  year_month                                             
89010 199101           24.261588  12.450976         4784.0   
      199102           16.875655  11.255854         4784.0   
      199103            0.000000   7.130000         4784.0   
      199104           51.209927   7.899268         4784.0   
      199105            0.000000   7.004146         4784.0   
...                          ...        ...            ...   
97635 201808            0.000000  18.950000         7370.0   
      201809            0.000000   4.200000         7370.0   
      201810            0.000000   3.453125         7370.0   
      201811            0.000000   3.415625         7370.0   
      201812            0.000000   3.393750         7370.0   

                  elevation_difference  zip_wspd  
ZCTA  year_month                                  
89010 199101              -3523.444444  0.878208  
      199102              -3412.000000  0.283772  
      199103                       NaN  1.129854  
      199104              -3777.333333  1.547416  
      199105                       NaN  1.658761  
...                                ...       ...  
97635 201808                       NaN  0.775684  
      201809                       NaN  0.599688  
      201810                       NaN  0.094439  
      201811                       NaN  0.914967  
      201812                       NaN  1.319343  

[547651 rows x 10 columns]

In [178]:
model_df2.to_csv(in_instrument + 'aggregated_draft_3.csv')

In [ ]:
model_df2

instrument  instrument_norm    bearing     distance  \
ZCTA  year_month                                                        
89019 199104        0.000261         0.006595   0.910217   185.292714   
      199107        0.000712         0.016371   5.377310   921.508343   
      199108        0.031378         0.717908   8.021400  1264.961710   
      199109        0.011910         0.275090  15.282616  2500.767308   
      199110        0.018859         0.453440  12.722091  2185.207157   
...                      ...              ...        ...          ...   
97635 201607        0.002553         0.036612  10.788656  1845.938462   
      201608        0.010405         0.152898  12.858277  2378.383437   
      201609        0.019936         0.275592  18.298730  3080.280585   
      201610        0.063282         1.022041   4.274471   443.813040   
      201611        0.011195         0.219072   2.608769   344.197776   

                  fire_wspd_x         acres    fire_lat     fire_lon  \
ZCTA  year_month                                                       
89019 199104         1.593638    207.908157   37.368992  -118.578197   
      199107         2.220699    733.828024  219.863048  -711.415978   
      199108         8.900734   3778.423931  325.520290 -1064.638511   
      199109         4.960312   7088.643047  617.857124 -2014.177914   
      199110         3.562893   6874.610214  510.727792 -1661.366877   
...                       ...           ...         ...          ...   
97635 201607         6.050591   1481.088247  529.755001 -1587.936493   
      201608         9.851180   3532.768287  611.579388 -1837.366152   
      201609        10.709470  12670.919912  894.747648 -2682.674963   
      201610         7.398554   8481.111023  204.069248  -604.084705   
      201611         1.582546   7721.865967  121.127814  -362.253242   

                  fire_days_in_mo    fire_wdir  fire_wspd_y   year  month  \
ZCTA  year_month                                                            
89019 199104             0.033333   356.694611     1.593638   1991      4   
      199107             0.354839   539.071667     2.220699  11946     42   
      199108             6.161290   434.880981     8.900734  17919     72   
      199109             9.066667  3306.518982     4.960312  33847    153   
      199110            13.096774  3704.847351     3.562893  27874    140   
...                           ...          ...          ...    ...    ...   
97635 201607             1.096774  1940.639033     6.050591  26208     91   
      201608             3.096774  3243.913956     9.851180  30240    120   
      201609             5.666667  5492.342896    10.709470  44352    198   
      201610             4.096774   277.967331     7.398554  10080     50   
      201611             2.800000   214.686996     1.582546   6048     33   

                  fire_pm25  fire_elevation  fire_pm25_norm  instrument_2  \
ZCTA  year_month                                                            
89019 199104          10.71          1630.0        0.121085      0.002795   
      199107          25.43         14253.0        0.287507      0.003166   
      199108          12.88         22620.0        0.145619      0.064362   
      199109          70.09         41888.0        0.792425      0.048156   
      199110          64.56         34152.0        0.729904      0.084305   
...                     ...             ...             ...           ...   
97635 201607          71.50          7937.0        0.808366      0.013238   
      201608          68.91          9088.0        0.779084      0.069987   
      201609         113.96         16837.0        1.288412      0.078531   
      201610          14.69          7816.0        0.166083      0.195177   
      201611          10.83          4938.0        0.122442      0.040414   

                  instrument_2_norm  
ZCTA  year_month                     
89019 199104               0.000293  
      199107 

In [ ]:
model_df2.to_csv(in_instrument + "aggregated_instrument_cols_useful_only.csv")

In [ ]:
model_df = fire_4d.drop(columns=["lat", "lon", "u", "v", "wind_lat_lon"])
model_df = model_df.drop(model_df[model_df["year"] > 2019].index)
model_df = model_df.drop(columns=["year", "month"])
model_df["fire_index"] = model_df["fire_index"].fillna(-1)
model_df["radius_mi"] = model_df["radius_mi"].fillna(-1)
model_df["treatment"] = model_df["treatment"].fillna(False)
model_df.fire_index = model_df.fire_index.astype(int)
model_df.fire_index = model_df.fire_index.astype(str)
model_df = model_df.drop_duplicates()
model_df["real_interaction"] = np.nan
model_df["real_distance"] = np.nan
model_df["interaction"] = np.nan
model_df["distance"] = np.nan
model_df["duration_days"] = model_df["duration_days"].fillna(-1)
model_df.groupby(["ZCTA", "year_month", "fire_index"]).first()
model_df